# 01_LangChain/DSpy Setup with Local Ollama Container Instance

This notebook demonstrates how to set up LangChain and DSpy to work with a local Ollama container instance. We will cover the installation of necessary packages, configuration, and connection to the local Ollama instance.

## Step 1: Install Necessary Packages

First, we need to install the required packages. Run the following command to install LangChain, DSpy, and other dependencies.

In [1]:
%pip install -q langchain langchain_community langchain_ollama dspy requests


Note: you may need to restart the kernel to use updated packages.


## Step 2: Import Packages

Next, we will import the necessary packages for our setup.

In [2]:
import dspy
from dspy import ChainOfThought
import requests

## Step 3: Configure Connection to Local Ollama Instance

We need to configure the connection to our local Ollama container instance. The following code sets up the connection.

### Step 3.1: Setup Ollama Docker Container Instance

*_Run the following step first. This step is only needed if the following step fails._*

#### Ollama
Ollama is a containerized environment for running and managing LLMs. It provides an API for interacting with the models.

#### Setup Instructions
1. Ensure Docker is installed and running on your machine.
2. Check if there is a Docker container instance 'ollama' that can be (re-)started.
3. _If no 'ollama' container exists_: Create and run a new Ollama container instance using the following command:
   ```
   docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
   ```
4. Verify the Ollama instance is running by accessing [http://localhost:11434/api/version](http://localhost:11434/api/version).
5. Verify programmatic connection by (re-)running the following cell:


In [3]:
OLLAMA_API_URL = "http://localhost:11434/api"

def get_ollama_version():
    response = requests.get(f"{OLLAMA_API_URL}/version")
    if response.status_code == 200:
        return response.json()
    else:
        return None

try:
    ollama_version = get_ollama_version()
    if ollama_version:
        print(f"Connected to Ollama version: {ollama_version}")
    else:
        print("Failed to connect to Ollama instance.")
except requests.exceptions.ConnectionError:
    print("Failed to connect to Ollama instance, is the Docker container running?")
    input("Press Enter to continue...")
    raise

Connected to Ollama version: {'version': '0.3.13'}


## Step 4: Minimal Documentation and Instructions

### LangChain
LangChain is a framework for building applications with large language models (LLMs). It provides tools and abstractions to simplify the development process.

### DSpy
DSpy is a data science library that offers various utilities for data manipulation, analysis, and visualization.


## Step 5: Demonstrate LangChain and DSpy Integration

In this step, we will demonstrate how to use LangChain and DSpy together in a practical example. We will use LangChain to generate text and DSpy to analyze the generated text.

### Example: Text Generation and Analysis

1. Use LangChain to generate text based on a prompt.
2. Use DSpy to analyze the generated text.

#### Generate Text with LangChain
We will use LangChain to generate text based on a given prompt.

In [4]:
def search_wikipedia(query: str, nr_articles=3) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=nr_articles)
    return [x['text'] for x in results]

In [5]:
# With new dspy version 2.5:
import dspy

lm = dspy.LM('ollama_chat/llama3.2', api_base='http://localhost:11434', api_key='')
rm = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')  # retrieval model
dspy.configure(lm=lm, rm=rm)

In [6]:
lm('Come up with 10 names for a song about infamous soccer players')

['Here are 10 potential song title ideas about infamous soccer players:\n\n1. "The Devil\'s in the Details" (about Diego Maradona)\n2. "Red Card Rhapsody" (about various players who have received red cards throughout history)\n3. "The Beautiful Game of Deceit" (about players like Zinedine Zidane and Marco van Basten, known for their skillful but sometimes questionable tactics)\n4. "Foul Play" (about players like Ronaldinho and Thierry Henry, infamous for their diving and other forms of cheating)\n5. "The King\'s Downfall" (about the rise and fall of famous soccer players like Cristiano Ronaldo or Lionel Messi)\n6. "Offside and Out" (about players who have been involved in high-profile controversies, such as Luis Suarez or Mario Balotelli)\n7. "The Beautiful Lie" (about the myth surrounding players like Pelé or Johan Cruyff, whose on-field performances were often shrouded in controversy)\n8. "Kicking It Old School" (about classic soccer players like George Best or Bobby Charlton, who we

In [7]:
ceo_role = rm("Chief Executive Officer", k=1)[0]['text']

In [8]:
print(ceo_role)

Chief executive officer | A chief executive officer (CEO) is the position of the most senior corporate officer, executive, leader or administrator in charge of managing an organization. CEOs lead a range of organizations, including public and private corporations, non-profit organizations and even some government organizations (e.g., Crown corporations). The CEO of a corporation or company typically reports to the board of directors and is charged with maximizing the value of the entity, which may include maximizing the share price, market share, revenues, or another element. In the non-profit and government sector, CEOs typically aim at achieving outcomes related to the organization's mission, such as reducing poverty, increasing literacy, etc. Titles also often given to the holder of CEO position include president, chief executive (CE), and managing director (MD), as well as representative director (RD) in Japan.


### References

- [LangChain Documentation](https://python.langchain.com/docs/)
- [DSpy Documentation](https://dspy.ai/tutorials/rag/)
- [LangChain and DSpy Integration](https://www.reddit.com/r/LangChain/comments/1cqexk6/thoughts_on_dspy/)

## Step 6: BasicQA DSpy Example with Signature Class

In this step, we will demonstrate a zero-shot example using DSpy and dspy.Signature class.


In [9]:
# Define the signature for the QA task
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Initialize the Predict module with the signature
generate_answer = dspy.Predict(BasicQA)

# Provide a question to the model
response = generate_answer(question="What is the capital of France?")

# Output the answer
print(f"Question: What is the capital of France?")
print(f"Answer: {response.answer}")

Question: What is the capital of France?
Answer: Paris


## Step 7: User Input for Software Project Idea

In this step, we will prompt the user to write a software project idea, send it to the LLM, and display the feedback, summary, and plan.

### Example: User Input and Feedback Loop

1. Prompt the user to write a software project idea.
2. Send the idea to the LLM and display the feedback, summary, and plan.
3. Implement a refinement loop to allow the user to provide additional input and receive updated feedback.

#### Prompt User for Software Project Idea
We will prompt the user to write a software project idea.

In [10]:
# 7.1 Get user input for software project idea, default to 
project_idea = input("Please write your software project idea: ") or "Create an AI software factory that generates software from project ideas, using step-by-step software processes that are implemented by LLM Agents."
print(project_idea)

Create an AI software factory that generates software from project ideas, using step-by-step software processes that are implemented by LLM Agents.


In [11]:
# Define a custom chain that uses a single-step prompt
# but instructs the model to produce the summary, strengths, and improvement_areas in one go.
initial_feedback = ChainOfThought(
    'idea -> feedback'
)

# Call the chain
feedback = initial_feedback(idea=project_idea)

# Print or handle the response
print(feedback)

Prediction(
    reasoning='The AI software factory will utilize Large Language Model (LLM) Agents to implement step-by-step software processes, ensuring efficient and effective project execution. This approach leverages the capabilities of LLMs in natural language processing, machine learning, and knowledge representation, allowing for the automation of complex software development tasks.',
    feedback='The proposed AI software factory will provide numerous benefits, including increased productivity, improved accuracy, and enhanced scalability. By automating repetitive and time-consuming tasks, developers can focus on higher-level creative work, leading to innovative solutions and faster project delivery. Additionally, the use of LLM Agents ensures consistency and reliability across projects, reducing the risk of human error.'
)


In [12]:
feedback_analyzer = ChainOfThought(
    'feedback -> strengths, improvements'
)


In [13]:
feedback_analysis = feedback_analyzer(feedback=feedback)
print(feedback_analysis)

Prediction(
    reasoning='The AI software factory will utilize Large Language Model (LLM) Agents to implement step-by-step software processes, ensuring efficient and effective project execution. This approach leverages the capabilities of LLMs in natural language processing, machine learning, and knowledge representation, allowing for the automation of complex software development tasks.',
    strengths='The proposed AI software factory will provide numerous benefits, including increased productivity, improved accuracy, and enhanced scalability. By automating repetitive and time-consuming tasks, developers can focus on higher-level creative work, leading to innovative solutions and faster project delivery. Additionally, the use of LLM Agents ensures consistency and reliability across projects, reducing the risk of human error.',
    improvements='There are no specific areas mentioned for improvement in the provided feedback. However, potential areas for future development could includ

In [14]:
# Idea iterator:
# Given an idea, with feedback from the UI, generate a new idea based on user input
project_idea_iterator = ChainOfThought(
    'project_idea, feedback -> improved_project_idea'
)


#### Implement Refinement Loop
We will implement a refinement loop to allow the user to provide additional input and receive updated feedback.

In [15]:
print(project_idea)

Create an AI software factory that generates software from project ideas, using step-by-step software processes that are implemented by LLM Agents.


In [16]:
project_idea_iterator(project_idea=project_idea, feedback='This idea is too big. I only want a minimum viable project to show my boss.')

Prediction(
    reasoning='The initial project idea is too broad and ambitious, requiring significant resources to implement. To make it more feasible, we need to scale down the scope of the project.',
    improved_project_idea='A minimum viable prototype (MVP) that demonstrates the core functionality of the AI software factory, focusing on a smaller subset of features and functionalities.'
)

In [17]:
while True:
    additional_input = input(f"""
                             Please provide additional input to refine your project idea (or type 'exit' to finish):
                             {project_idea}
                              """)
    if not additional_input.strip():
        new_idea = project_idea
        break
    
    new_idea = project_idea_iterator(project_idea=project_idea, feedback=additional_input)
    project_idea = new_idea.improved_project_idea
    
    print(project_idea)

In [18]:
new_idea


'Create an AI software factory that generates software from project ideas, using step-by-step software processes that are implemented by LLM Agents.'

## Step 8: Write a Short Style Guide

In this step, we will write a short style guide summarizing the preferred coding style for the project, including type hinting, docstrings, and testability.

### Style Guide

#### Type Hinting
- Use type hints for function arguments and return values.
- Example:
  ```python
  def add(a: int, b: int) -> int:
      return a + b
  ```

#### Docstrings
- Use docstrings to document functions, classes, and modules.
- Follow the Google style for docstrings.
- Example:
  ```python
  def add(a: int, b: int) -> int:
      """Add two integers.

      Args:
          a (int): The first integer.
          b (int): The second integer.

      Returns:
          int: The sum of the two integers.
      """
      return a + b
  ```

#### Testability
- Write testable code by following the principles of modularity and separation of concerns.
- Use dependency injection to make code more testable.
- Write unit tests for all functions and classes.
- Example:
  ```python
  def add(a: int, b: int) -> int:
      return a + b

  def test_add():
      assert add(1, 2) == 3
  ```

## Step 9: Add an Agentic/Smolagents @tool to Review a File for a Certain Style Guide

In this step, we will add an agentic/smolagents @tool to review a file for a certain style guide.

In [19]:
from smoltools import tool
import os

@tool
def review_file(file_path: str, style_guide: str) -> str:
    """Review a file for a certain style guide.

    Args:
        file_path (str): The path to the file to review.
        style_guide (str): The style guide to review against.

    Returns:
        str: The review results.
    """
    with open(file_path, 'r') as file:
        content = file.read()

    # Placeholder for actual review logic
    review_results = f"Reviewing {file_path} against {style_guide}...\n"
    review_results += "No issues found."

    return review_results

## Step 10: Include Prompt Templates to Execute the Reviews

In this step, we will include prompt templates to execute the reviews.

In [20]:
prompt_template = """
You are a code reviewer. Your task is to review the following file for adherence to the specified style guide.

File Path: {file_path}
Style Guide: {style_guide}

Please provide a detailed review, highlighting any issues and suggesting improvements.
"""

def generate_review_prompt(file_path: str, style_guide: str) -> str:
    return prompt_template.format(file_path=file_path, style_guide=style_guide)

## Step 11: Demonstrate Example Review Results on a File

In this step, we will demonstrate example review results on a file.

In [21]:
example_file_path = "example.py"
example_style_guide = "PEP 8"

review_results = review_file(example_file_path, example_style_guide)
print(review_results)

Reviewing example.py against PEP 8...
No issues found.
